In [1]:
from SimPEG import *
from simpegEM1D import *
from scipy.constants import mu_0
import numpy as np
frequency = np.array([382, 1822, 7970, 35920, 130100], dtype=float)
hz = get_vertical_discretization_frequency(frequency, sigma_background=0.01)
mesh1D = set_mesh_1d(hz)
depth = -mesh1D.gridN[:-1]
LocSigZ = -mesh1D.gridCC

FDsurvey = EM1DSurveyFD(
    rx_location = np.array([0., 0., 100.+30.]),
    src_location = np.array([0., 0., 100.+30.]),
    field_type = 'secondary',
    rx_type = 'ppm',
    src_type = 'VMD',
    offset = np.ones(5)*7.86,
    topo = np.r_[0., 0., 100.],
    depth = depth,
    frequency = frequency
)

sig_half = 1e-3
sig_blk = sig_half*50.
chi_half = 0.
expmap = maps.ExpMap(mesh1D)
sig  = np.ones(FDsurvey.n_layer)*sig_half
blk_ind = (-50>LocSigZ) & (-80<LocSigZ)
sig[blk_ind] = sig_blk
# sig[-150>LocSigZ] = 0.1
m_true = np.r_[np.log(sig)]

In [3]:
sim = EM1DFMSimulation(
    mesh1D, survey=FDsurvey, sigmaMap=expmap,
    chi= np.zeros(FDsurvey.n_layer)
)
  
d_true = sim.dpred(m_true)
m0 = np.ones_like(m_true) * np.log(sig_half)
d_0 = sim.dpred(m0)

In [ ]:
FDsurvey.dtrue = d_true

std = 0.05
floor = 0.
np.random.seed(1)
uncert = std*abs(d_true)+floor
noise = std*d_true*np.random.randn(d_true.size)
d_obs = d_true + noise

dataObj = data.Data(FDsurvey, dobs=d_obs, noise_floor=uncert)

doi, _= sim.depth_of_investigation_christiansen_2012(uncert)
delta = sim.get_threshold(uncert)

In [ ]:
print (doi, delta)

In [ ]:
from SimPEG import regularization

In [ ]:

dmisfit = data_misfit.L2DataMisfit(dataObj, sim)
dmisfit.W = 1./(abs(data_obj.dobs)*std+floor)

reg = regularization.Sparse(
    mesh1D, mapping=maps.IdentityMap(mesh1D)
)
p=0.
qx, qz = 2., 2.
reg.norms = np.c_[p, qx, qz, 0.]
IRLS = directives.update_IRLS(
    max_irls_iterations=20, minGNiter=1, fix_Jmatrix=True, coolingRate=2, 
    betaSearch=False,
    chifact_start = 1.
)
opt = optimization.ProjectedGNCG(maxIter = 40)
opt.maxIterLS = 5
invProb = inverse_problem.BaseInvProblem(dmisfit, reg, opt)
beta = directives.BetaSchedule(coolingFactor=2, coolingRate=2)
betaest = directives.BetaEstimate_ByEig(beta0_ratio=1.)
target = directives.TargetMisfit()
sense = directives.UpdateSensitivityWeights(threshold=delta)
# inv = Inversion.BaseInversion(invProb, directiveList=[beta,betaest,target])
inv = inversion.BaseInversion(invProb, directiveList=[IRLS, betaest])

opt.LSshorten = 0.5
opt.remember('xc')
mopt = inv.run(m0)

In [ ]:
doi, act_ind = sim.depth_of_investigation_christiansen_2012(uncert)

In [ ]:
print (doi)

In [ ]:
%pylab inline
fig, ax = subplots(1,1, figsize=(5, 8))
Utils1D.plotLayer(sig, mesh1D, showlayers=False)
Utils1D.plotLayer(expmap*mopt, mesh1D, showlayers=False, **{'color':'r'})
Utils1D.plotLayer(expmap*invProb.l2model, mesh1D, showlayers=False, **{'color':'b', 'lw':1})

In [ ]:
print (doi)

In [ ]:
delta = prob.get_threshold(uncert)

In [ ]:
print (delta)

In [ ]:
# !python run_em1d.py

In [ ]:
fig, axes = subplots(1,1, figsize = (7,5))
axes.plot(FDsurvey.frequency, -invProb.dpred[0:FDsurvey.n_frequency], 'k.-')
axes.plot(FDsurvey.frequency, -invProb.dpred[FDsurvey.n_frequency:], 'b.-')
axes.plot(FDsurvey.frequency, -FDsurvey.dobs[0:FDsurvey.n_frequency], 'ko')
axes.plot(FDsurvey.frequency, -FDsurvey.dobs[FDsurvey.n_frequency:], 'bo')
# axes.plot(FDsurvey.frequency, uncert[0:FDsurvey.n_frequency], 'k*')
# axes.plot(FDsurvey.frequency, uncert[FDsurvey.n_frequency:], 'r*')
axes.set_xscale('log')

In [ ]:
from scipy.linalg.blas import cgemm
import numpy as np

In [ ]:
fig, ax = subplots(1,1, figsize=(5, 8))
Utils1D.plotLayer(sig, mesh1D, showlayers=True)